In [5]:
import torch
import numpy as np
from sklearn.decomposition import PCA 

In [6]:
from MyModel_EEC import RoBERTa
torch.hub._validate_not_a_forked_repo=lambda a,b,c: True

In [7]:
import os
os.environ['CUDA_VISIBLE_DEVICES']='1'

model = RoBERTa()

Downloading: "https://github.com/pytorch/fairseq/archive/master.zip" to /home/weicheng/.cache/torch/hub/master.zip
2023-05-02 22:16:25 | INFO | fairseq.file_utils | loading archive file http://dl.fbaipublicfiles.com/fairseq/models/roberta.base.tar.gz from cache at /home/weicheng/.cache/torch/pytorch_fairseq/37d2bc14cf6332d61ed5abeb579948e6054e46cc724c7d23426382d11a31b2d6.ae5852b4abc6bf762e0b6b30f19e741aa05562471e9eb8f4a6ae261f04f9b350
2023-05-02 22:16:28 | INFO | fairseq.tasks.masked_lm | dictionary: 50264 types
2023-05-02 22:16:36 | INFO | fairseq.models.roberta.model | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 25, 'log_format': 'json', 'log_file': None, 'aim_repo': None, 'aim_run_hash': None, 'tensorboard_logdir': None, 'wandb_project': None, 'azureml_logging': False, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': True, 'memory_efficient_fp16': True, 'fp16_no_flatten_grads': False, 'fp16_init_sca

Model Loaded.


In [8]:

model.load_state_dict(torch.load('/home/weicheng/NLU/NLU_debias/saves/model_pt_epoch_test.pt', map_location='cpu'))

In [ ]:
roberta = torch.hub.load('pytorch/fairseq', 'roberta.large')

In [ ]:
text_male = 'he son his him father man boy himself male brother sons fathers men boys males brothers uncle uncles nephew nephews'
text_female = 'she daughter hers her mother woman girl herself female sister daughters mothers women girls females sisters aunt aunts niece nieces'

In [ ]:
text_male = 'he son his him father man boy himself male brother'
text_female = 'she daughter hers her mother woman girl herself female sister'

In [ ]:
text_male = text_male.split(' ')
text_female = text_female.split(' ')

new_ls=[ ]
for i in range(len(text_male)):
    new_ls.append(text_male[i] + ' ' + text_female[i])

# print(new_ls)
new_ls = ' '.join(new_ls)
print(new_ls)
# tokens = roberta.encode(text)
# print(roberta.decode(tokens))


In [ ]:
tokens = roberta.encode(new_ls)
print(roberta.decode(tokens))
last_layer_features = roberta.extract_features(tokens)
print(last_layer_features.shape)

In [ ]:
last_layer_features[0:1, 1:-1].shape

In [ ]:
def doPCA(pairs, num_components=10):
    matrix = []
    for i in range(0, len(pairs)-1, 2):
        a = pairs[i]
        b = pairs[i+1]
    # for a, b in pairs:
        center = (a+b)/2
        matrix.append(a-center)
        matrix.append(b-center)
    matrix = np.array(matrix)
    pca = PCA(n_components=num_components)
    pca.fit(matrix)
    return pca

In [ ]:
pca = doPCA(last_layer_features[0, 1:-1].detach().numpy(), 2)

In [ ]:
pca.components_.shape

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
## TODO: 
# def ss_calculation(text_words):
    

In [ ]:
text_words = 'joy sad pride'
tokens = roberta.encode(text_words)
print(tokens)
# print(roberta.decode(tokens))
print([roberta.decode(t.unsqueeze(0)) for t in tokens])
print(roberta.decode(tokens))
last_layer_features = roberta.extract_features(tokens)

In [ ]:
last_layer_features.shape

In [ ]:
new_word = last_layer_features[:, 1:-1, :]

In [ ]:
x_axis, y_axis = pca.components_

In [ ]:
print(x_axis.shape)
print(new_word.shape)

In [ ]:
proj_x = torch.matmul(new_word, torch.tensor(x_axis).unsqueeze(1))/torch.norm(torch.tensor(x_axis))

In [ ]:
proj_x

In [ ]:
proj_y = torch.matmul(new_word, torch.tensor(y_axis).unsqueeze(1))/torch.norm(torch.tensor(y_axis))

In [ ]:
proj_y

In [ ]:
plt.scatter(proj_x.item(), proj_y.item(), )